## Introdução 

Esta lição abordará: 
- O que é chamada de função e seus casos de uso 
- Como criar uma chamada de função usando OpenAI 
- Como integrar uma chamada de função em uma aplicação 

## Objetivos de Aprendizagem 

Após concluir esta lição, você saberá como e entenderá: 

- O propósito de usar chamada de função 
- Configurar Chamada de Função usando o Serviço OpenAI 
- Projetar chamadas de função eficazes para o caso de uso da sua aplicação 


## Entendendo Chamadas de Função

Para esta lição, queremos construir um recurso para nossa startup de educação que permita aos usuários usar um chatbot para encontrar cursos técnicos. Recomendaremos cursos que se encaixem no nível de habilidade, cargo atual e tecnologia de interesse deles.

Para completar isso, usaremos uma combinação de:
 - `OpenAI` para criar uma experiência de chat para o usuário
 - `Microsoft Learn Catalog API` para ajudar os usuários a encontrar cursos com base na solicitação do usuário
 - `Function Calling` para pegar a consulta do usuário e enviá-la para uma função para fazer a solicitação da API.

Para começar, vamos ver por que gostaríamos de usar chamadas de função em primeiro lugar:

print("Mensagens na próxima solicitação:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # obter uma nova resposta do GPT onde ele pode ver a resposta da função


print(second_response.choices[0].message)


### Por que Chamar Funções

Se você completou qualquer outra lição deste curso, provavelmente entende o poder de usar Modelos de Linguagem Grande (LLMs). Esperançosamente, você também consegue ver algumas de suas limitações.

Chamar Funções é um recurso do Serviço OpenAI projetado para resolver os seguintes desafios:

Formatação Inconsistente de Respostas:
- Antes de chamar funções, as respostas de um modelo de linguagem grande eram não estruturadas e inconsistentes. Os desenvolvedores precisavam escrever códigos complexos de validação para lidar com cada variação na saída.

Integração Limitada com Dados Externos:
- Antes desse recurso, era difícil incorporar dados de outras partes de uma aplicação em um contexto de chat.

Ao padronizar os formatos de resposta e permitir integração perfeita com dados externos, chamar funções simplifica o desenvolvimento e reduz a necessidade de lógica adicional de validação.

Os usuários não conseguiam obter respostas como "Qual é o clima atual em Estocolmo?". Isso porque os modelos estavam limitados ao tempo em que os dados foram treinados.

Vamos ver o exemplo abaixo que ilustra esse problema:

Digamos que queremos criar um banco de dados de dados de estudantes para que possamos sugerir o curso certo para eles. Abaixo temos duas descrições de estudantes que são muito semelhantes nos dados que contêm.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Queremos enviar isso para um LLM para analisar os dados. Isso pode ser usado posteriormente em nossa aplicação para enviar para uma API ou armazenar em um banco de dados.

Vamos criar dois prompts idênticos nos quais instruímos o LLM sobre quais informações estamos interessados:


Queremos enviar isso para um LLM para analisar as partes que são importantes para o nosso produto. Assim, podemos criar dois prompts idênticos para instruir o LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Após criar esses dois prompts, enviaremos eles para o LLM usando `openai.ChatCompletion`. Armazenamos o prompt na variável `messages` e atribuímos o papel de `user`. Isso é para imitar uma mensagem de um usuário sendo escrita para um chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Agora podemos enviar ambas as solicitações para o LLM e examinar a resposta que recebemos.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Mesmo que os prompts sejam os mesmos e as descrições sejam semelhantes, podemos obter formatos diferentes da propriedade `Grades`.

Se você executar a célula acima várias vezes, o formato pode ser `3.7` ou `3.7 GPA`.

Isso ocorre porque o LLM recebe dados não estruturados na forma do prompt escrito e também retorna dados não estruturados. Precisamos ter um formato estruturado para saber o que esperar ao armazenar ou usar esses dados.

Ao usar chamadas funcionais, podemos garantir que recebemos dados estruturados de volta. Ao usar chamadas funcionais, o LLM na verdade não chama ou executa nenhuma função. Em vez disso, criamos uma estrutura para o LLM seguir em suas respostas. Em seguida, usamos essas respostas estruturadas para saber qual função executar em nossas aplicações.


![Diagrama de Fluxo de Chamada de Função](../../../../translated_images/br/Function-Flow.083875364af4f4bb.webp)


Podemos então pegar o que é retornado pela função e enviar isso de volta para o LLM. O LLM então responderá usando linguagem natural para responder à consulta do usuário.


### Casos de Uso para chamadas de função

**Chamar Ferramentas Externas**  
Chatbots são ótimos para fornecer respostas a perguntas dos usuários. Usando chamadas de função, os chatbots podem usar mensagens dos usuários para completar certas tarefas. Por exemplo, um estudante pode pedir ao chatbot para "Enviar e-mail para meu instrutor dizendo que preciso de mais ajuda com este assunto". Isso pode fazer uma chamada de função para `send_email(to: string, body: string)`

**Criar Consultas de API ou Banco de Dados**  
Os usuários podem encontrar informações usando linguagem natural que é convertida em uma consulta formatada ou requisição de API. Um exemplo disso pode ser um professor que solicita "Quem são os alunos que completaram a última tarefa" que poderia chamar uma função chamada `get_completed(student_name: string, assignment: int, current_status: string)`

**Criar Dados Estruturados**  
Os usuários podem pegar um bloco de texto ou CSV e usar o LLM para extrair informações importantes dele. Por exemplo, um estudante pode converter um artigo da Wikipedia sobre acordos de paz para criar flash cards de IA. Isso pode ser feito usando uma função chamada `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Criando Sua Primeira Chamada de Função

O processo de criar uma chamada de função inclui 3 etapas principais:  
1. Chamar a API de Completações de Chat com uma lista de suas funções e uma mensagem do usuário  
2. Ler a resposta do modelo para executar uma ação, ou seja, executar uma função ou chamada de API  
3. Fazer outra chamada para a API de Completações de Chat com a resposta da sua função para usar essa informação para criar uma resposta para o usuário.


![Fluxo de uma Chamada de Função](../../../../translated_images/br/LLM-Flow.3285ed8caf4796d7.webp)


### Elementos de uma chamada de função 

#### Entrada dos Usuários 

O primeiro passo é criar uma mensagem do usuário. Isso pode ser atribuído dinamicamente pegando o valor de uma entrada de texto ou você pode atribuir um valor aqui. Se esta for sua primeira vez trabalhando com a API de Conclusões de Chat, precisamos definir o `role` e o `content` da mensagem. 

O `role` pode ser `system` (criando regras), `assistant` (o modelo) ou `user` (o usuário final). Para chamadas de função, atribuiremos isso como `user` e uma pergunta de exemplo. 


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Criando funções.

A seguir, definiremos uma função e os parâmetros dessa função. Usaremos apenas uma função aqui chamada `search_courses`, mas você pode criar várias funções.

**Importante**: As funções são incluídas na mensagem do sistema para o LLM e serão contabilizadas na quantidade de tokens disponíveis que você tem.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definições** 

A estrutura de definição da função possui múltiplos níveis, cada um com suas próprias propriedades. Aqui está uma divisão da estrutura aninhada:

**Propriedades da Função no Nível Superior:**

`name` - O nome da função que queremos que seja chamada.

`description` - Esta é a descrição de como a função funciona. Aqui é importante ser específico e claro.

`parameters` - Uma lista de valores e formato que você deseja que o modelo produza em sua resposta.

**Propriedades do Objeto Parameters:**

`type` - O tipo de dado do objeto parameters (geralmente "object").

`properties` - Lista dos valores específicos que o modelo usará para sua resposta.

**Propriedades Individuais dos Parâmetros:**

`name` - Implicitamente definido pela chave da propriedade (ex.: "role", "product", "level").

`type` - O tipo de dado deste parâmetro específico (ex.: "string", "number", "boolean").

`description` - Descrição do parâmetro específico.

**Propriedades Opcionais:**

`required` - Um array listando quais parâmetros são necessários para que a chamada da função seja concluída.


### Fazendo a chamada da função  
Após definir uma função, agora precisamos incluí-la na chamada para a API de Conclusão de Chat. Fazemos isso adicionando `functions` à requisição. Neste caso, `functions=functions`.  

Também há uma opção para definir `function_call` como `auto`. Isso significa que deixaremos o LLM decidir qual função deve ser chamada com base na mensagem do usuário, em vez de atribuí-la nós mesmos.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Agora vamos analisar a resposta e ver como ela está formatada:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Você pode ver que o nome da função foi chamado e, a partir da mensagem do usuário, o LLM conseguiu encontrar os dados para preencher os argumentos da função.


## 3.Integrando Chamadas de Função em uma Aplicação. 


Depois de testarmos a resposta formatada do LLM, agora podemos integrá-la em uma aplicação. 

### Gerenciando o fluxo 

Para integrar isso em nossa aplicação, vamos seguir os seguintes passos: 

Primeiro, vamos fazer a chamada aos serviços da OpenAI e armazenar a mensagem em uma variável chamada `response_message`. 


In [ ]:
response_message = response.choices[0].message

Agora vamos definir a função que chamará a API do Microsoft Learn para obter uma lista de cursos:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Como uma boa prática, veremos então se o modelo deseja chamar uma função. Depois disso, criaremos uma das funções disponíveis e a associaremos à função que está sendo chamada.  
Em seguida, pegaremos os argumentos da função e os mapearemos para os argumentos do LLM.

Por fim, adicionaremos a mensagem de chamada da função e os valores que foram retornados pela mensagem `search_courses`. Isso fornece ao LLM todas as informações necessárias para  
responder ao usuário usando linguagem natural.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Agora enviaremos a mensagem atualizada para o LLM para que possamos receber uma resposta em linguagem natural em vez de uma resposta formatada em JSON da API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Desafio de Código 

Ótimo trabalho! Para continuar seu aprendizado sobre OpenAI Function Calling, você pode construir: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - Mais parâmetros da função que podem ajudar os aprendizes a encontrar mais cursos. Você pode encontrar os parâmetros de API disponíveis aqui: 
 - Criar outra chamada de função que receba mais informações do aprendiz, como sua língua nativa 
 - Criar tratamento de erros quando a chamada da função e/ou chamada da API não retornar cursos adequados 


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte autorizada. Para informações críticas, recomenda-se tradução profissional humana. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
